In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os

In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "train_id": "hierarchical_single",
    'sample_size': 1
})


In [5]:

job_path = "/mnt/disks/data/fma/trains"

# In[15]:

train_path = os.path.join(job_path,args.train_id)

# In[16]:

base_path = os.path.join(args.root_dir,"fma")

# In[17]:

models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")

# In[18]:

metadata_file = os.path.join(train_path,"metadata.json")


categories_labels_path = os.path.join(train_path,"labels.json")


In [6]:


def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp




In [7]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True



In [8]:
create_dir(train_path)

True

## Load genres file. Contains relationships beetwen genres

In [9]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [10]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [11]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


In [12]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'))

In [13]:
tracks_df = tracks_df.sample(frac=args.sample_size)

In [14]:
tracks_df.sample(20)

,track_id,track_title,valid_genre
5649,10657,Absorbed III,"['1', '15', '38']"
48533,72582,100 Miles,"['76', '113']"
17624,28857,Eat Dust,"['1', '19', '38']"
87556,132127,Lost Keys,"['15', '495']"
63762,97705,LAmentos,"['27', '70', '181']"
24531,38771,Favela feat. Kit,"['15', '21', '468']"
33920,52361,Lose Yourself at the Nymphaeum,"['12', '314']"
102807,153359,02 - l'hymne,['64']
43632,65747,Friday Night Knife Fight,['10']
93703,140108,The Restoration [35:49—42:33],"['25', '38', '97']"


In [15]:
tracks_df.valid_genre.values

array(["['53', '58']", "['2', '8', '214']", "['15', '38', '41']", ...,
       "['66', '111']", "['15']", "['322']"], dtype=object)

In [16]:
tracks_df.track_title

8664                                     Axis Mundi
5521                          El-Ouache-Ouel-Ghorba
32590                                        Wander
77967                                  Le telephoné
85648    La Màgia de l'Ànima Impossible - desfilada
                            ...                    
81032      Carol Annes Crunch - Studio Tape Track 5
26640                                 Vacío Estelar
42217                                       Grifted
16772                                   s6c6wr3m7x2
50839                                       Prelude
Name: track_title, Length: 104186, dtype: object

In [17]:
## Get complete genre structure
def get_all_structure(estrutura,df_genres):
    ## Get structure from df_genres
    def get_all_structure_from_df(estrutura,df_genres,structure=[]):
        if estrutura == 0:
            return structure
        else:
            structure.append(estrutura)
            get_all_structure_from_df(df_genres[df_genres["genre_id"]==int(estrutura)].parent.values[0],df_genres,structure)
            return structure
    
    return get_all_structure_from_df(estrutura,df_genres,structure=[])
    

In [18]:
# tracks_df['valid_genre'] = tracks_df.track_genres.apply(lambda x: x.strip('][').split(', ') if x != '[]' else None)
tracks_df['valid_genre'] = tracks_df.valid_genre.apply(lambda x: ast.literal_eval(x))

In [19]:
tracks_df['valid_genre']

8664               [53, 58]
5521            [2, 8, 214]
32590          [15, 38, 41]
77967                   [1]
85648         [15, 42, 296]
                ...        
81032                  [18]
26640    [12, 31, 109, 619]
42217             [66, 111]
16772                  [15]
50839                 [322]
Name: valid_genre, Length: 104186, dtype: object

In [20]:
tracks_df['last_genre_id'] = tracks_df.valid_genre.apply(lambda x:x[-1] if x != None else None)

In [21]:
tracks_df.sample(20)

,track_id,track_title,valid_genre,last_genre_id
78482,120020,Drift,"[2, 311]",311
26087,40799,hoaaaaa!!!,"[15, 296]",296
25834,40499,He was an ok man but he's dead now,"[76, 224]",224
7744,13561,The Dao of Sun Zoo,[21],21
60016,91647,Recueillement,"[94, 98]",98
94422,141181,Stay (Teknique Remix) (Featuring Jeff Barringer),"[10, 362]",362
65298,100380,Dark Ophrys,"[1, 30, 38, 41, 247]",247
39894,60687,Alone and Forsaken,[12],12
43052,65040,Anxiety Ejection,"[15, 468]",468
49019,73319,Untitled Edit,"[1, 30, 38, 41, 247]",247


In [22]:
# tracks_df.dropna(inplace=True)

In [23]:
tracks_df

,track_id,track_title,valid_genre,last_genre_id
8664,14758,Axis Mundi,"[53, 58]",58
5521,10512,El-Ouache-Ouel-Ghorba,"[2, 8, 214]",214
32590,50627,Wander,"[15, 38, 41]",41
77967,119401,Le telephoné,[1],1
85648,129741,La Màgia de l'Ànima Impossible - desfilada,"[15, 42, 296]",296
...,...,...,...,...
81032,123468,Carol Annes Crunch - Studio Tape Track 5,[18],18
26640,41499,Vacío Estelar,"[12, 31, 109, 619]",619
42217,64029,Grifted,"[66, 111]",111
16772,27634,s6c6wr3m7x2,[15],15


In [24]:
tracks_df['full_genre_id'] = tracks_df.last_genre_id.apply(lambda x: get_all_structure(x,genres_df)[::-1])

In [25]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[2, 86, 173]        4
[20, 65, 189]       4
[9, 651, 493]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [26]:
tracks_df.columns

Index(['track_id', 'track_title', 'valid_genre', 'last_genre_id',
       'full_genre_id'],
      dtype='object')

In [27]:
tracks_df = tracks_df[['track_id','full_genre_id']]

In [28]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[2, 86, 173]        4
[20, 65, 189]       4
[9, 651, 493]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [29]:
tracks_df.full_genre_id.info

<bound method Series.info of 8664          [12, 58]
5521      [2, 92, 214]
32590         [38, 41]
77967          [38, 1]
85648        [15, 296]
             ...      
81032       [1235, 18]
26640    [2, 130, 619]
42217    [12, 25, 111]
16772             [15]
50839         [5, 322]
Name: full_genre_id, Length: 104186, dtype: object>

In [30]:
labels_size = tracks_df.full_genre_id.apply(lambda x: len(x))

In [31]:
labels_size = labels_size.max()

In [32]:
labels_size

5

In [33]:
import pandas as pd
import os

### Parse of label to structure

In [34]:
### Function for parse label to sctructure of hierarhical scheme

def parse_label(label,label_size=5):
    # label = label.split('-')
    # preencher com 0 no caso de haver menos de 5 níveis
    labels = np.zeros(label_size,dtype=int)
    for i, label in enumerate(label):
        if i == 5:
            break
        # Aqui você pode fazer a conversão do label em um índice inteiro usando um dicionário ou outro método
        # Neste exemplo, estou apenas usando a posição da label na lista como índice
        labels[i] = label
    return labels

In [35]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[2, 86, 173]        4
[20, 65, 189]       4
[9, 651, 493]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [36]:
parsed_labels = tracks_df.full_genre_id.apply(lambda x: parse_label(x))

In [37]:
def convert_label_to_string(x,level=2):
    return '-'.join([str(value) for value in x[:level]])

In [38]:
tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))
tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))
tracks_df['labels_3'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=3))
tracks_df['labels_4'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=4))
tracks_df['labels_5'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=5))

  0%|          | 0/104186 [00:00<?, ?it/s]

/tmp/ipykernel_201279/310633048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))


  0%|          | 0/104186 [00:00<?, ?it/s]

/tmp/ipykernel_201279/310633048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))


  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

In [39]:
tracks_df.labels_1.value_counts()

38      22066
12      21710
15      18493
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [40]:
tracks_df['labels_1'].value_counts()

38      22066
12      21710
15      18493
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [41]:
tracks_df = tracks_df[tracks_df['labels_1'].isin(['3','14'])]

In [42]:
tracks_df.to_csv(os.path.join(train_path,"tracks.csv"),index=False)

In [43]:
tracks_df = pd.read_csv(os.path.join(train_path,"tracks.csv"))

In [44]:
categories_df = pd.DataFrame({'level5':tracks_df.labels_5.unique()})

In [45]:
categories_df['level1'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:1]))
categories_df['level2'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:2]))
categories_df['level3'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:3]))
categories_df['level4'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:4]))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [46]:
categories_df

,level5,level1,level2,level3,level4
0,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0
1,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0
2,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0
3,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0
4,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0


In [47]:
def get_labels_name(x,genres_df):
    levels = 5
    full_name = []
    last_level = 0
    genre_root = ""
    for genre in x.split('-'):
        genre_df = genres_df[genres_df['genre_id'] == int(genre)]
        if genre_df.empty:
            genre_name = genre_root 
        else:
            genre_name = genre_df.title.values.tolist()[0]
            genre_root = genre_name
        
        full_name.append(genre_name)
    full_name = '>'.join(full_name)
        
    return full_name
    # return genres_df[genres_df['genre_id'] == int(x)].title.values.tolist()[0]

In [48]:
categories_df['level5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x,genres_df))

In [49]:
categories_df

,level5,level1,level2,level3,level4,level5_name
0,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0,Soul-RnB>Funk>Funk>Funk>Funk
1,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0,Blues>Gospel>Gospel>Gospel>Gospel
2,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0,Soul-RnB>Disco>Disco>Disco>Disco
3,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0,Blues>Blues>Blues>Blues>Blues
4,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0,Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB


In [51]:
def __create_labels__(categories_df):
    data = {
        "label1": {},
        "label2": {},
        "label3": {},
        "label4": {},
        "label5": {},
        "label1_inverse": [],
        "label2_inverse": [],
        "label3_inverse": [],
        "label4_inverse": [],
        "label5_inverse": [],
        "label1_name": {},
        "label2_name": {},
        "label3_name": {},
        "label4_name": {},
        "label5_name": {},
    }

    idx = 0
    
    for id_x, cat in enumerate(set(categories_df.level1.values.tolist())):
        data['label1'][cat] = idx
        data['label1_inverse'].append(cat)
        data['label1_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level2.values.tolist())):
        data['label2'][cat] = idx
        data['label2_inverse'].append(cat)
        data['label2_count'] = idx + 1
        idx+=1
        
    for id_x, cat in enumerate(set(categories_df.level3.values.tolist())):
        data['label3'][cat] = idx
        data['label3_inverse'].append(cat)
        data['label3_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level4.values.tolist())):
        data['label4'][cat] = idx
        data['label4_inverse'].append(cat)
        data['label4_count'] = idx + 1
        idx+=1
        
    for idx, cat in enumerate(set(categories_df.level5.values.tolist())):
        data['label5'][cat] = idx
        data['label5_inverse'].append(cat)
        data['label5_count'] = idx + 1
        idx+=1
        
    for cat5,cat1,cat2,cat3,cat4,name5 in categories_df.values:
        
        name1 = '>'.join(name5.split('>')[:1])
        name2 = '>'.join(name5.split('>')[:2])
        name3 = '>'.join(name5.split('>')[:3])
        name4 = '>'.join(name5.split('>')[:4])
        
        
        data['label1_name'][cat1] = name1
        data['label2_name'][cat2] = name2
        data['label3_name'][cat3] = name3
        data['label4_name'][cat4] = name4
        data['label5_name'][cat5] = name5
        
    return data

In [58]:
with open(categories_labels_path, 'w+') as f:
    f.write(json.dumps(__create_labels__(categories_df)))

In [52]:
labels  = __create_labels__(categories_df)

In [57]:
labels['label4']

{'3-0-0-0': 12,
 '14-0-0-0': 13,
 '3-567-0-0': 14,
 '14-19-0-0': 15,
 '14-11-0-0': 16}